In [16]:
from data_store import DataStore
from data_store.data_containers import Experiments, Dataset
from data_store.storage import GCSStorage
import glob
import os
import yaml
import pandas as pd
import glob

import nest_asyncio
nest_asyncio.apply()

In [3]:
# %%


gs = GCSStorage(
    # endpoint="http://0.0.0.0:4443", 
    path=f"gs://cape-ml-projects-data/data_stores"
)
ds = DataStore(name="dinov2", storage=gs)

Populating queue...
Download started...
queue fully populated. Found 1 files to download
Processed 0 files in 5.3 s. In queue: 225
Processed 1 files in 10.4 s. In queue: 215
Downloaded 1 files in 15.0 s


In [4]:
ds['experiments/a100x4'] = Experiments(description="Experiments run on a single A100x4 machine")

Starting upload of 1 files
Finished 1 uploads in 15.0 s


In [5]:
exp = ds['experiments/a100x4']

In [7]:
with open("/data/output/config.yaml") as fp:
    config = yaml.safe_load(fp)

In [12]:
meta = pd.json_normalize(config).iloc[0].to_dict()

In [13]:
e = exp.create(experiment_id="DIN-121", description="Run on the 2m dataset", **meta)

In [21]:
e.upload(["/data/output/config.yaml", "/data/output/training_metrics.json"], root_path="/data/output/")

Starting upload of 2 files
Finished 2 uploads in 15.0 s


In [15]:
e.tree()

data_stores/dinov2/experiments/a100x4/DIN-121/
└── config.yaml


In [18]:
files = [x for x in glob.glob("/data/output/eval/**/*", recursive=True) if os.path.isfile(x)]
e.upload(files, root_path="/data/output")

Starting upload of 10 files
Completed 0/10 uploads in 15.0 s
Finished 10 uploads in 30.1 s


In [22]:
e.tree()

data_stores/dinov2/experiments/a100x4/DIN-121/
├── config.yaml
├── training_metrics.json
└── eval/
    ├── training_112499/
    │   └── teacher_checkpoint.pth
    ├── training_12499/
    │   └── teacher_checkpoint.pth
    ├── training_124999/
    │   └── teacher_checkpoint.pth
    ├── training_24999/
    │   └── teacher_checkpoint.pth
    ├── training_37499/
    │   └── teacher_checkpoint.pth
    ├── training_49999/
    │   └── teacher_checkpoint.pth
    ├── training_62499/
    │   └── teacher_checkpoint.pth
    ├── training_74999/
    │   └── teacher_checkpoint.pth
    ├── training_87499/
    │   └── teacher_checkpoint.pth
    └── training_99999/
        └── teacher_checkpoint.pth
